### Evaluating grubbs test for outliers to remove spikes

In [1]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 166
%matplotlib qt5

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pynapple as nap

import matplotlib.pyplot as plt



#### synthetic data - numpy arrays

In [3]:
from outlier_detection import grubbs_single
N = 10000
t = np.linspace(0,100,N)
y = np.random.randn(N)
params = dict(mode='classic', alpha=0.05)
# control run
grubbs_single(y, **params)

False

### synthetic fluorescence data

In [12]:
import iblphotometry.preprocessing
import iblphotometry.synthetic

df, times = iblphotometry.synthetic.synthetic101(30, 1000, 0.1)
F = df['raw_calcium'].values
F = F + np.random.randn(F.shape[0]) * 0.1

# random injection of N noise spikes of random lengths
N = np.random.randint(0, 100) # random number of spikes
inds = np.random.randint(0, F.shape[0], N) # at random locations
L = np.random.randint(1, 10, size=N) # of random lengths
sig = 2

# inserting
for i, (ix, l) in enumerate(zip(inds,L)):
    sign = np.array([-1,1])[np.random.randint(2)]
    F[ix:ix+l] += np.random.rand()+1 * sig * sign

# nap
F = nap.Tsd(t=df['times'].values, d=F)
fig, axes = plt.subplots()
axes.plot(F,lw=0.5)


/home/georg/code/ibl-photometry/src/iblphotometry/synthetic.py:20: DeprecationWarning: scipy.signal.ricker is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  ric = scipy.signal.ricker(int(fs * 4), 8)


In [15]:
# run it
from outlier_detection import remove_outliers
params = dict(w_size=1000, alpha=0.05, w=25)
F_or = remove_outliers(F, **params)
fig, axes = plt.subplots()
axes.plot(F, lw=2,color='r',)
axes.plot(F_or, lw=1,color='k')

### on real data

In [ ]:
from utils import * # don't

from one.api import ONE

one = ONE(base_url="https://alyx.internationalbrainlab.org")